## Chart the RSEI Score and Pounds post-treatment values for selected chemicals
Compare all USA facilities in RSEI with facilities offered a reporting exemption.

Chemicals selected (ChemicalNumber in parentheses):

'1,2,4-Trichlorobenzene', (564)

'1,2,4-Trimethylbenzene', (575)

'Benzene', (51)

'Hexachlorobenzene', (321)

'Mercury', (359)

'Mercury compounds', (360)

'Polychlorinated biphenyls ', (474)

'Polycyclic aromatic compounds' (609)

‘Ethylene oxide’, (293)

Group the first four benzene-related chemicals in the Benzene chart.

Combine Mercury and Mercury compounds



In [ ]:
# Install our codebase 
# !pip install ECHO_modules >&/dev/null;
%pip install git+https://github.com/edgi-govdata-archiving/ECHO_modules@neighborhoods >&/dev/null;
%pip install geopandas >&/dev/null;

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

def make_chart(graph_title, data_column, y_axis_label, df):
    #set seaborn plotting aesthetics
    sns.set(style='white')

    #create grouped bar chart

    fig, ax = plt.subplots(figsize=(10,10))
    try:
        g = sns.barplot(x='SubmissionYear', y=data_column, hue='Who', data=df)
        g.set_title(graph_title)
        ax.set_ylabel(y_axis_label)
        return g
    except TypeError as te:
        print("TypeError: {}".format(str(te)))
        return None

The file All-submissions-elements-4-chemicals.csv is produced by the chemicals-by-fac.py program.

The file Exemptions-facility-chemicals is produced by running the RSEI.ipynb notebook, with the Exemptions-FullList.csv facilities (FRSID List), Media=[1,2] (air releases).

On Google Colab download these two files from this Google Drive: https://drive.google.com/drive/folders/1a-h_EhDKuKFBt0EdNSCi6a3OxQ2YyBT-?usp=drive_link.
Then upload them into the base file folder on the left of the screen.

In [ ]:
# usa_filename = '/home/steve/workspace/EDGI/RSEI/CSVs/All-submissions-elements-4-chemicals.csv'
# chemicals_filename = '/home/steve/workspace/EDGI/RSEI/CSVs/Exemptions-facility_chemicals.csv'

usa_filename = '/content/All-submissions-elements-4-chemicals.csv'
chemicals_filename = '/content/Exemptions-facility_chemicals.csv'

In [ ]:
from ECHO_modules.get_data import get_echo_data

usa = pd.read_csv(usa_filename)
# usa.set_index('Year', inplace=True)
usa = usa[usa['Media'].isin([1,2])]
usa['Who'] = 'USA'
# usa.rename(columns={'USA' : data_column}, inplace=True)

exemption_chemicals = pd.read_csv(chemicals_filename)
# usa.set_index('Year', inplace=True)
exemption_chemicals['Who'] = 'Exemptions'
# usa.rename(columns={'USA' : data_column}, inplace=True)

# Add the ChemicalNumber to the exemption_chemicals
sql = 'select "Chemical", "ChemicalNumber" from "chemical_data_rsei_v2312"'
chem_df = get_echo_data(sql)

exemption_chemicals = pd.merge(exemption_chemicals, chem_df, on="Chemical", how="inner")

In [ ]:

chemical_nums = {
                 'Lead' : [346, 347],
                 'Arsenic' : [39, 40],
                 'Chromium' : [152, 153],
                 'Nickel' : [405, 406],
                 'Cadmium' : [108, 109],
                 'Mercury' : [359, 360], 
                 'Ethylene Oxide' : [293],
                 'Chloroprene' : [145],
                 'Dioxins' : [614],
                 'Furans' : [113, 191, 617],
                 'Benzene' : [51, 321, 564, 575],
                 'Toluene' : [187, 188, 269, 270, 271, 549, 550, 551, 552, 628],
                 'Hydrochloride' : [35, 219, 247, 248, 258, 457, 458, 533, 554],
                 'Arsenic' : [39, 40],
                 'Hydrogen Fluoride' : [259, 333]
                 }

rules_chemicals = {
                   "HON Rule" : ['Ethylene Oxide', 'Chloroprene', 'Dioxins', 'Furans'],
                   "MATS" : ['Mercury', 'Lead', 'Arsenic', 'Chromium', 'Nickel', 'Cadmium'],
                   "Sterilizer Rule" : ['Ethylene Oxide',],
                   "Rubber Tire Rule" : ['Benzene',],
                   "Copper Rule" : ['Lead', 'Arsenic', 'Mercury', 'Benzene', 'Toluene', 'Hydrochloride'],
                   "Iron and Steel Rule" : ['Mercury', 'Hydrochloride', 'Dioxins', 'Furans'],
                   "Lime Rule" : ['Mercury', 'Hydrochloride', 'Dioxins', 'Furans'],
                   "Coke Ovens Rule" : ['Benzene', 'Mercury', 'Lead', 'Arsenic'],
                   "Taconite Rule" : ['Hydrogen Fluoride'],
                  }


In [ ]:
for rule_key, rule_value in rules_chemicals.items():
    fac_rule = exemption_chemicals[exemption_chemicals['Standard'] == rule_key]
    usa_other = usa[~usa['FacilityID'].isin(fac_rule['FacilityID'])]
    to_chart_exempt = {}
    to_chart_usa = {}
    for chem_name in rule_value:
        exempt_chem = fac_rule[fac_rule['ChemicalNumber'].isin(chemical_nums[chem_name])]
        usa_other_chem = usa_other[usa_other['ChemicalNumber'].isin(chemical_nums[chem_name])]
        exemptions_count = len(exempt_chem['FacilityID'].unique())
        usa_count = len(usa_other_chem['FacilityID'].unique())
        exemptions_to_chart = exempt_chem.groupby(['SubmissionYear'], as_index=False).agg({'Score' : 'sum', 'PoundsReleased' : 'sum', 'PoundsPT' : 'sum'})
        exemptions_to_chart['Score'] /= exemptions_count
        exemptions_to_chart['PoundsReleased'] /= exemptions_count
        exemptions_to_chart['PoundsPT'] /= exemptions_count
        exemptions_to_chart['Who'] = f'{rule_key} facilities'
        # to_chart_exempt[chem_name] = exemptions_to_chart
        usa_to_chart = usa_other_chem.groupby(['SubmissionYear'], as_index=False).agg({'Score' : 'sum', 'PoundsReleased' : 'sum', 'PoundsPT' : 'sum'})
        if usa_count > 0:
            usa_to_chart['Score'] /= usa_count
            usa_to_chart['PoundsReleased'] /= usa_count
            usa_to_chart['PoundsPT'] /= usa_count
            usa_to_chart['Who'] = f'Other {chem_name} facilities'

            to_chart = pd.concat([usa_to_chart, exemptions_to_chart], axis=0)
            to_chart['SubmissionYear'] = to_chart['SubmissionYear'].astype(int)

            data_column = 'Score'
            y_axis = 'Average RSEI Score - per facility'
            graph_title = f'{chem_name} - {rule_key} Facilities and All Other USA Facilities'
            graph_title += f'\n{exemptions_count} producers of {chem_name} subject to {rule_key}, {usa_count} in all other RSEI facilities'
            display(make_chart(graph_title=graph_title, data_column=data_column, y_axis_label=y_axis, df=to_chart))
    
            data_column = 'PoundsPT'
            graph_title = f'{chem_name} - {rule_key} Facilities and All Other USA Facilities'
            y_axis = 'Average Pounds Post-treatment - per facility'
            graph_title += f'\n{exemptions_count} producers of {chem_name} subject to {rule_key}, {usa_count} in all other RSEI facilities'
            display(make_chart(graph_title=graph_title, data_column=data_column, y_axis_label=y_axis, df=to_chart))
        elif exemptions_count > 0:
            to_chart = exemptions_to_chart
            to_chart['SubmissionYear'] = to_chart['SubmissionYear'].astype(int)
            
            data_column = 'Score'
            y_axis = 'Average RSEI Score - per facility'
            graph_title = f'{chem_name} - {rule_key} Facilities - No Other RSEI Facilities for This Rule/Chemical'
            graph_title += f'\n{exemptions_count} producers of {chem_name} subject to {rule_key}'
            display(make_chart(graph_title=graph_title, data_column=data_column, y_axis_label=y_axis, df=to_chart))
    
            data_column = 'PoundsPT'
            graph_title = f'{chem_name} - {rule_key} Facilities - No Other RSEI Facilities for This Rule/Chemical'
            y_axis = 'Average Pounds Post-treatment - per facility'
            graph_title += f'\n{exemptions_count} producers of {chem_name} subject to {rule_key}'
            display(make_chart(graph_title=graph_title, data_column=data_column, y_axis_label=y_axis, df=to_chart))
        # to_chart_usa[chem_name] = usa_to_chart

### Below is an older cell producing graphs, that does not distinguish the different rules (Standard) that the facilities are following

In [ ]:
chemicals = {51 : 'Benzene', 
             359 : 'Mercury', 
             474 : 'Polychorinated biphenyls', 
             609 : 'Polycyclic aromatic compounds', 
             293 : 'Ethylene oxide'}

for key, value in chemicals.items():
    if key == 51:
        chemical_usa = usa[usa['ChemicalNumber'].isin([51, 321, 564, 575])]
        chemical_exemptions = exemption_chemicals[exemption_chemicals['Chemical'].isin(['1,2,4-Trichlorobenzene', 
                                                                      '1,2,4-Trimethylbenzene', 
                                                                      'Hexachlorobenzene',
                                                                      'Benzene'])]
    elif key == 359:
        chemical_usa = usa[usa['ChemicalNumber'].isin([359, 360])]
        chemical_exemptions = exemption_chemicals[exemption_chemicals['Chemical'].isin(['Mercury', 
                                                                      'Mercury compounds'])]
    else:
        chemical_usa = usa[usa['ChemicalNumber'] == key]
        chemical_exemptions = exemption_chemicals[exemption_chemicals['Chemical'] == value]

    exemptions_count = len(chemical_exemptions['FacilityID'].unique())
    usa_count = len(chemical_usa['FacilityID'].unique())
    print(f'{value} - exemptions_count={exemptions_count}, usa_count={usa_count}')

    exemptions_to_chart = chemical_exemptions.groupby(['SubmissionYear'], as_index=False).agg({'Score' : 'sum', 'PoundsReleased' : 'sum', 'PoundsPT' : 'sum'})
    exemptions_to_chart['Score'] /= exemptions_count
    exemptions_to_chart['PoundsReleased'] /= exemptions_count
    exemptions_to_chart['PoundsPT'] /= exemptions_count
    usa_to_chart = chemical_usa.groupby(['SubmissionYear'], as_index=False).agg({'Score' : 'sum', 'PoundsReleased' : 'sum', 'PoundsPT' : 'sum'})
    usa_to_chart['Score'] /= usa_count
    usa_to_chart['PoundsReleased'] /= usa_count
    usa_to_chart['PoundsPT'] /= usa_count

    usa_to_chart['Who'] = 'USA'
    exemptions_to_chart['Who'] = 'Exemptions'

    to_chart = pd.concat([usa_to_chart, exemptions_to_chart], axis=0)
    to_chart['SubmissionYear'] = to_chart['SubmissionYear'].astype(int)

    data_column = 'Score'
    graph_title = f'{value} - Exemptions and All Other USA Facilities'
    y_axis = 'Average RSEI Score - per facility'
    display(make_chart(graph_title=graph_title, data_column=data_column, y_axis_label=y_axis, df=to_chart))
    
    data_column = 'PoundsPT'
    graph_title = f'{value} - Exemptions and All Other USA Facilities'
    y_axis = 'Average Pounds Post-treatment - per facility'
    display(make_chart(graph_title=graph_title, data_column=data_column, y_axis_label=y_axis, df=to_chart))